In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=(
    SparkSession.builder.appName("SparkIntroduction").getOrCreate())
spark
# sc=spark.spark.Context

In [2]:
df_data=spark.read.csv("final_data.csv",header=True,inferSchema=True,mode="DROPMALFORMED")
df_data.show()

+---------------------+-------------------+--------------------+--------------+------+-----------+---------+------+--------------------+-----------------+-----+-----+------------------+------------------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|trans_date_trans_time|             cc_num|            merchant|      category|   amt|      first|     last|gender|              street|             city|state|  zip|               lat|              long|city_pop|                 job|       dob|           trans_num| unix_time|         merch_lat|        merch_long|is_fraud|
+---------------------+-------------------+--------------------+--------------+------+-----------+---------+------+--------------------+-----------------+-----+-----+------------------+------------------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|  2019-08-11 19:38:33|  

In [3]:
df_data.printSchema()
df_data.count()

root
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)



19302

In [4]:
data=df_data.distinct()

In [5]:
data.count()

19302

In [7]:
from pyspark.sql.functions import avg
avg_spent= df_data.groupBy("gender").agg(avg("amt")).alias("avg_amount")
avg_spent.show()

+------+------------------+
|gender|          avg(amt)|
+------+------------------+
|     F|271.71298766157497|
|     M|330.50721562156156|
+------+------------------+



In [8]:
from pyspark.sql.functions import count, year, month
trans = df_data.groupBy(year("trans_date_trans_time").alias("year"),month("trans_date_trans_time").alias("month")).agg(count("trans_num").alias("transaction_count"))
trans = trans.orderBy("year", "month")
trans.show()

+----+-----+-----------------+
|year|month|transaction_count|
+----+-----+-----------------+
|2019|    1|              809|
|2019|    2|              755|
|2019|    3|              847|
|2019|    4|              755|
|2019|    5|              764|
|2019|    6|              816|
|2019|    7|              775|
|2019|    8|              822|
|2019|    9|              812|
|2019|   10|              806|
|2019|   11|              741|
|2019|   12|             1293|
|2020|    1|              604|
|2020|    2|              605|
|2020|    3|              817|
|2020|    4|              661|
|2020|    5|              910|
|2020|    6|              925|
|2020|    7|              788|
|2020|    8|              872|
+----+-----+-----------------+
only showing top 20 rows



In [9]:
fraud_df=df_data.filter(df_data.is_fraud == 1)
fraud_df.show()
fraud_df.count()

+---------------------+----------------+--------------------+-------------+-------+-------+------+------+--------------------+-------------+-----+-----+-------+---------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|trans_date_trans_time|          cc_num|            merchant|     category|    amt|  first|  last|gender|              street|         city|state|  zip|    lat|     long|city_pop|                 job|       dob|           trans_num| unix_time|         merch_lat|        merch_long|is_fraud|
+---------------------+----------------+--------------------+-------------+-------+-------+------+------+--------------------+-------------+-----+-----+-------+---------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|  2019-01-02 01:06:37|   4613314721966|fraud_Rutherford-...|  grocery_pos| 281.06|  Jason|Murphy|     M|542 Steve Curve S...|C

9651

In [11]:
from pyspark.sql.functions import sum
fraud = fraud_df.groupBy("trans_date_trans_time").agg(sum("is_fraud").alias("fraud_count")).orderBy("trans_date_trans_time").distinct()
fraud.show()
fraud.count()

+---------------------+-----------+
|trans_date_trans_time|fraud_count|
+---------------------+-----------+
|  2019-01-12 23:32:25|          1|
|  2019-02-08 22:34:34|          1|
|  2019-02-12 22:03:43|          1|
|  2019-02-22 23:38:20|          1|
|  2019-02-28 22:02:41|          1|
|  2019-04-26 15:45:56|          1|
|  2019-05-16 22:03:23|          1|
|  2019-05-18 22:21:21|          1|
|  2019-05-22 00:17:46|          1|
|  2019-06-03 03:05:46|          1|
|  2019-07-10 22:53:15|          1|
|  2019-09-20 02:42:03|          1|
|  2019-10-18 23:19:26|          1|
|  2019-12-12 01:06:43|          1|
|  2020-01-27 23:17:38|          1|
|  2020-03-24 00:33:35|          1|
|  2019-04-12 01:27:31|          1|
|  2019-05-29 23:57:34|          1|
|  2019-06-09 03:11:29|          1|
|  2019-07-27 22:32:30|          1|
+---------------------+-----------+
only showing top 20 rows



9646

In [12]:
from pyspark.sql.functions import year,current_date
df_age = data.select("dob","first","last", (year(current_date())- year("dob")).alias("age_transaction")).orderBy("first").distinct()
df_age.show()

+----------+--------+---------+---------------+
|       dob|   first|     last|age_transaction|
+----------+--------+---------+---------------+
|1969-12-22| Monique| Robinson|             56|
|1967-01-24|Kathleen|    Heath|             58|
|1988-03-09|Jennifer|    Banks|             37|
|1956-05-15|Nicholas|  Osborne|             69|
|1953-04-19| Michael|Rodriguez|             72|
|1958-09-10| Douglas|   Willis|             67|
|1966-12-21|   Jerry| Robinson|             59|
|1930-02-28|   Frank|      Key|             95|
|1979-01-26|  Tamara| Martinez|             46|
|1990-10-19|    Lisa|     Lowe|             35|
|1973-06-09| Michael| Williams|             52|
|1969-01-14|   James|   Bishop|             56|
|1997-12-26|   Karen|   Warren|             28|
|1988-09-19|  Brooke|Rodriguez|             37|
|1949-04-24|Michelle|  Russell|             76|
|1985-04-15|   Alice|     Kemp|             40|
|1930-08-13| Jessica|   Jordan|             95|
|1990-04-25|   Tracy|   Conway|         

In [13]:
df_age.count()

999